In [1]:
import os
from PIL import Image 
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import numpy as np
import time
import cv2 

import pytube

In [2]:
video_url = 'https://www.youtube.com/watch?v=yt2YKtjIwrk' # paste here your Youube videos' url

In [3]:
def download_youtube(url):
    youtube = pytube.YouTube(url)
    video = youtube.streams.first()
    video = video.download("./")
    return video

## Load Yolo

In [4]:
def load_yolo(config_path,weights_path,confidence, nms):
    
    
    class_names = []
    with open("cars.names","r") as f:
        class_names = [line.strip() for line in f.readlines()]
    colors = np.random.uniform(0,255,size=(len(class_names),3))
    net = cv2.dnn.readNet(weights_path,config_path)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    model = cv2.dnn_DetectionModel(net)
    model.setInputParams(size=(416, 416), scale=1/256)
    return model, colors, class_names

### IMAGE

In [5]:
def image_preds(image,confidence,nms):
    img = cv2.imread(image)
    img = cv2.resize(img, None, fx=0.4,fy=0.4)
    model, colors, class_names = load_yolo("yolov4-tiny-custom_best.weights","yolov4-tiny-custom.cfg",.4,.6)
    classes,scores, boxes = model.detect(img, confidence,nms)
    
    for (classid, score, box) in zip(classes, scores, boxes):
        color = colors[int(classid) % len(colors)]
        label = "%s : %f" % (class_names[classid[0]], score)
        cv2.rectangle(img, box, color, 2)
        cv2.putText(img, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imshow("test_img",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
image_preds("yolo/city1.jpg",.3,.3)

FileNotFoundError: [Errno 2] No such file or directory: 'cars.names'

### Video

In [3]:
def video_preds(video):
    vc = cv2.VideoCapture(video)
    while(True): 
        (grabbed, frame) = vc.read()
        if not grabbed:
            exit()

        start = time.time()
        classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
        end = time.time()

        start_drawing = time.time()
        for (classid, score, box) in zip(classes, scores, boxes):
            color = COLORS[int(classid) % len(COLORS)]
            label = "%s : %f" % (class_names[classid[0]], score)
            cv2.rectangle(frame, box, color, 2)
            cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        end_drawing = time.time()

        fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
        cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.imshow("detections", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    vc.release() 
    # Destroy all the windows 
    cv2.destroyAllWindows()        

In [4]:
video_preds("chicagotest.mp4")

IndexError: list index out of range

### Webcam

In [3]:
def webcam_preds():
    vc = cv2.VideoCapture(0)
    while(True): 
        (grabbed, frame) = vc.read()
        if not grabbed:
            exit()

        start = time.time()
        classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
        end = time.time()

        start_drawing = time.time()
        for (classid, score, box) in zip(classes, scores, boxes):
            color = COLORS[int(classid) % len(COLORS)]
            label = "%s : %f" % (class_names[classid[0]], score)
            cv2.rectangle(frame, box, color, 2)
            cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        end_drawing = time.time()

        fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
        cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.imshow("detections", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    vc.release() 
    # Destroy all the windows 
    cv2.destroyAllWindows() 
    

In [4]:
webcam_preds()